<a href="https://colab.research.google.com/github/maxysio/DS-PRJ2-DataStructures/blob/master/05-BlockChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import hashlib
from time import gmtime, strftime
import time

In [0]:
class Block:
    def __init__(self, timestamp, data, previous_hash):
      if data is None or data == '':
        print('Invalid Data for Block Chain')
        return
        
      self.timestamp = timestamp
      self.data = data
      self.previous_hash = previous_hash
      self.hash = self.calc_hash()

    def calc_hash(self):
      sha = hashlib.sha256()
      sha.update(self.data.encode('utf-8'))
      return sha.hexdigest()

In [0]:
class BlockChain:
  def __init__(self):
    self.blocks = []
    self.head = None
    
  def prepend(self, data):
    # Create a new block
    new_block = Block(time.gmtime(), data, None)
    self.blocks.insert(0, new_block)

    if self.head is None:
      # If no head, that would mean its an empty BlockChain
      self.head = new_block
      return
    
    # Assign the new blocks hash to the previous hash of the head and move head to the new block
    self.head.previous_hash = new_block.hash
    self.head = new_block

  def append(self, data):
    # Create a new block
    new_block = Block(time.gmtime(), data, None)

    if self.size()>0:
      # Assign the hash from the end to the new blocks previous hash
      new_block.previous_hash = self.blocks[-1].hash

    self.blocks.append(new_block)

    if self.head is None:
      self.head = new_block
      return

  def search(self, data):
    # create a hash of the data
    sha = hashlib.sha256()
    sha.update(data.encode('utf-8'))
    data_hash = sha.hexdigest()

    # Run through the list to find the block object
    for b in self.blocks:
      if b.hash == data_hash:
        return b
    print('{} not found in BlockChain'.format(data))
    return None
    
  def remove(self, data):
    # If the size is 0, then nothing to remove
    if self.size() == 0:
      return
    
    block = self.search(data)
    if block:
      # If data is found in the block chain,
      # Check if the size of the block chain is 1, then its the only object
      if self.size() == 1:
        self.blocks = []
        return

      # Find the block before and after it and link them
      
      prev_block = None
      next_block = None
      
      # For the block before, travel through the list, 
      # check block's previous hash against hash of the data in the block in the list
      for b in self.blocks:
        if block.previous_hash == b.hash:
          prev_block = b
          break
      
      # For the block after, travel through the list,
      # check hash of current block's data against the previous hash of the block in the list
      for b in self.blocks:
        if b.previous_hash == block.hash:
          next_block = b
          break
      
      if next_block and prev_block:
        # Block to be deleted in the middle
        next_block.previous_hash = prev_block.hash
      elif next_block:
        # Block to be deleted is in the begining
        next_block.previous_hash = None

      # If Block to be deleted is at the end, no action item

      # Finally remove the block from the list
      self.blocks.remove(block)

  def pop(self):
    # Remove the last item from the Block Chain
    
    # Check if the block chain is empty
    if self.size()==0:
      print('BlockChain is empty')
      return

    block = self.blocks[-1]

    # Remove
    self.remove(block.data)

    return block

  def insert(self, data, pos):
    # if block chain is empty, just add it irrespective of position
    if self.size() == 0:
      self.append(data)
      return

    # if pos is 1, then its the begining, so prepend it
    if pos == 1:
      self.prepend(data)
      return

    # if position is greater than the size of the BlockChain, add it to the end
    if pos > self.size():
      self.append(data)
      return

    # Create a new block
    new_block = Block(time.gmtime(), data, None)

    # Get the block from the list for that position and the previous one
    next_block = self.blocks[pos-1]
    prev_block = self.blocks[pos-2]

    # assign the has correctly
    new_block.previous_hash = next_block.previous_hash
    next_block.previous_hash = new_block.hash
    
    # insert new block into the list
    self.blocks.insert(pos-1, new_block)

  def size(self):
    return len(self.blocks)

TEST CODE BELOW

In [0]:
bc = BlockChain()
bc.append('Block1')
bc.append('Block2')
bc.append('Block3')
bc.append('Block5')

In [32]:
bc.size()

4

In [33]:
for b in bc.blocks:
  print('Data: {}'.format(b.data), 'Curr Hash: {}'.format(b.hash), 'Prev hash: {}'.format(b.previous_hash))

Data: Block1 Curr Hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103 Prev hash: None
Data: Block2 Curr Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380 Prev hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103
Data: Block3 Curr Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704 Prev hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380
Data: Block5 Curr Hash: 543ef0b11f17bef0e05f7335930915b763e0c227e58455b6d2c9759698ef686a Prev hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704


In [0]:
bc.prepend('PBlock6')
bc.prepend('PBlock7')
bc.prepend('PBlock8')

In [35]:
for b in bc.blocks:
  print('Data: {}'.format(b.data), 'Curr Hash: {}'.format(b.hash), 'Prev hash: {}'.format(b.previous_hash))

Data: PBlock8 Curr Hash: 426aca398b0c462b95ad8833fc75b44273038ced80fadb330a868110448b96f2 Prev hash: None
Data: PBlock7 Curr Hash: 229fa73566ee3d5e9ab73718c12c88bd388d7cd33de283e0d582c7ec5dbed106 Prev hash: 426aca398b0c462b95ad8833fc75b44273038ced80fadb330a868110448b96f2
Data: PBlock6 Curr Hash: d5fe465bb0fed6b682f14523c1f96790ff88a62805b51874afdf7264fcd258a2 Prev hash: 229fa73566ee3d5e9ab73718c12c88bd388d7cd33de283e0d582c7ec5dbed106
Data: Block1 Curr Hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103 Prev hash: d5fe465bb0fed6b682f14523c1f96790ff88a62805b51874afdf7264fcd258a2
Data: Block2 Curr Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380 Prev hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103
Data: Block3 Curr Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704 Prev hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380
Data: Block5 Curr Hash: 543ef0b11f17bef0e05f7335930915b763e0c227e58

In [0]:
bc.insert('Block65', 3)
bc.insert('Block66', 3)
bc.insert('Block67', 3)

In [37]:
for b in bc.blocks:
  print('Data: {}'.format(b.data), 'Curr Hash: {}'.format(b.hash), 'Prev hash: {}'.format(b.previous_hash))

Data: PBlock8 Curr Hash: 426aca398b0c462b95ad8833fc75b44273038ced80fadb330a868110448b96f2 Prev hash: None
Data: PBlock7 Curr Hash: 229fa73566ee3d5e9ab73718c12c88bd388d7cd33de283e0d582c7ec5dbed106 Prev hash: 426aca398b0c462b95ad8833fc75b44273038ced80fadb330a868110448b96f2
Data: Block67 Curr Hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3 Prev hash: 229fa73566ee3d5e9ab73718c12c88bd388d7cd33de283e0d582c7ec5dbed106
Data: Block66 Curr Hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216 Prev hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3
Data: Block65 Curr Hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01 Prev hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216
Data: PBlock6 Curr Hash: d5fe465bb0fed6b682f14523c1f96790ff88a62805b51874afdf7264fcd258a2 Prev hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01
Data: Block1 Curr Hash: 40e9b17a3391b5f461b2b96a2e5810a885f08834

In [0]:
bc.remove('PBlock8')
bc.remove('PBlock7')
bc.remove('PBlock6')

In [39]:
for b in bc.blocks:
  print('Data: {}'.format(b.data), 'Curr Hash: {}'.format(b.hash), 'Prev hash: {}'.format(b.previous_hash))

Data: Block67 Curr Hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3 Prev hash: None
Data: Block66 Curr Hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216 Prev hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3
Data: Block65 Curr Hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01 Prev hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216
Data: Block1 Curr Hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103 Prev hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01
Data: Block2 Curr Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380 Prev hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103
Data: Block3 Curr Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704 Prev hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380
Data: Block5 Curr Hash: 543ef0b11f17bef0e05f7335930915b763e0c227e58

In [40]:
ret_block = bc.pop()
ret_block.data

'Block5'

In [41]:
for b in bc.blocks:
  print('Data: {}'.format(b.data), 'Curr Hash: {}'.format(b.hash), 'Prev hash: {}'.format(b.previous_hash))

Data: Block67 Curr Hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3 Prev hash: None
Data: Block66 Curr Hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216 Prev hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3
Data: Block65 Curr Hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01 Prev hash: 82be552aee2a51fc9bc3227314844a9a89d0d4455846b8e84545c7ec013e2216
Data: Block1 Curr Hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103 Prev hash: 5581c0c73982976806aa457f542e48ba18886aaa9c60a8eb9b192a2757e6db01
Data: Block2 Curr Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380 Prev hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103
Data: Block3 Curr Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704 Prev hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380


In [42]:
bc.append(None)

Invalid Data for Block Chain
